# WorkFlow
### Imports
### Load the data
### Cleanning
### FE
### Data.corr()
### Analytics
### Preproccessing
### Decomposition
### Feature Selection
### Modelling
### Random Search
### Gird Search

## Imports

In [1]:
import random
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import torch,torchvision
from torch.nn import *
from torch.optim import *
# Preproccessing
from sklearn.preprocessing import (
    StandardScaler,
    RobustScaler,
    MinMaxScaler,
    MaxAbsScaler,
    OneHotEncoder,
    Normalizer,
    Binarizer
)
# Decomposition
from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA
# Feature Selection
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import RFECV
from sklearn.feature_selection import SelectFromModel
# Model Eval
from sklearn.compose import make_column_transformer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score,train_test_split
from sklearn.metrics import mean_absolute_error,mean_squared_error,accuracy_score,precision_score,f1_score,recall_score
# Models
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LogisticRegression,LogisticRegressionCV
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor,AdaBoostRegressor,VotingRegressor,BaggingRegressor,RandomForestRegressor
from sklearn.svm import SVR
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import ExtraTreesRegressor
from catboost import CatBoost,CatBoostRegressor
from xgboost import XGBRegressor,XGBRFRegressor
from flaml import AutoML
# Other
import pickle
import wandb

PROJECT_NAME = 'House-Prices-Advanced-Regression-Techniques-V9'
device = 'cuda'
np.random.seed(21)
random.seed(21)
torch.manual_seed(21)

### Funtions

In [2]:
def make_submission(model,name):
    data = pd.read_csv('./data/test.csv')
    ids = data['Id']
    str_cols = []
    int_cols = []
    for col_name,num_of_missing_rows,dtype in zip(list(data.columns),data.isna().sum(),data.dtypes):
        if dtype == object:
            str_cols.append(col_name)
        else:
            int_cols.append(col_name)
    for str_col in str_cols:
        data,idx,labels_and_int_index,new_data = object_to_int(data,str_col)
    nan_cols = []
    for col_name,num_of_missing_rows,dtype in zip(list(data.columns),data.isna().sum(),data.dtypes):
        if num_of_missing_rows > 0:
            nan_cols.append(col_name)
    for nan_col in nan_cols:
        data[nan_col].fillna(data[nan_col].median(),inplace=True)
    preds = model.predict(data)
    df = pd.DataFrame({'Id':ids,'SalePrice':preds})
    df.to_csv(f'./submission/{name}.csv',index=False)

In [3]:
def valid(model,X,y,valid=False):
    preds = model.predict(X)
    if valid:
        results = {
            'val mean_absolute_error':mean_absolute_error(y_true=y,y_pred=preds),
            'val mean_squared_error':mean_squared_error(y_true=y,y_pred=preds),
        }
    else:
        results = {
            'mean_absolute_error':mean_absolute_error(y_true=y,y_pred=preds),
            'mean_squared_error':mean_squared_error(y_true=y,y_pred=preds),
        }
    return results

In [4]:
def train(model,X_train,X_test,y_train,y_test,name):
    wandb.init(project=PROJECT_NAME,name=name)
    model.fit(X_train,y_train)
    wandb.log(valid(model,X_train,y_train))
    wandb.log(valid(model,X_test,y_test,True))
    make_submission(model,name)
    return model

In [5]:
def object_to_int(data,col):
    data_col = data[col].to_dict()
    idx = -1
    labels_and_int_index = {}
    for data_col_vals in data_col.values():
        if data_col_vals not in labels_and_int_index.keys():
            idx += 1
            labels_and_int_index[data_col_vals] = idx
    new_data = []
    for data_col_vals in data_col.values():
        new_data.append(labels_and_int_index[data_col_vals])
    data[col] = new_data
    return data,idx,labels_and_int_index,new_data

In [6]:
def fe(data,col,quantile_max_num=0.99,quantile_min_num=0.05):
    max_num = data[col].quantile(quantile_max_num)
    min_num = data[col].quantile(quantile_min_num)
    print(max_num)
    print(min_num)
    data = data[data[col] < max_num]
    data = data[data[col] > min_num]
    return data

In [7]:
def decomposition(X,pca=False,kernal_pca=False):
    if pca:
        pca = PCA()
        X = pca.fit_transform(X)
    if kernal_pca:
        kernal_pca = KernelPCA()
        X = kernal_pca.fit_transform(X)
    return X

In [8]:
def feature_selection_prep_data(model,X,y,select_from_model=False,variance_threshold=False,select_k_best=False,rfecv=False):
    if select_from_model:
        transform = SelectFromModel(estimator=model.fit(X, y))
        X = transform.transform(X)
    if variance_threshold:
        transform = VarianceThreshold()
        X = transform.fit_transform(X)
    if select_k_best:
        X = SelectKBest(chi2, k='all').fit_transform(X, y)
    if rfecv:
        X = RFECV(model, step=1, cv=5).fit(X, y)
        X = X.transform(X)
    return X

In [9]:
def prep_data(X,transformer):
    mct = make_column_transformer(
        (transformer,list(X.columns)),
        remainder='passthrough'
    )
    X = mct.fit_transform(X)
    return X

## Load the data

In [10]:
data = pd.read_csv('./data/train.csv')

In [11]:
preproccessings = [StandardScaler,RobustScaler,MinMaxScaler,MaxAbsScaler,OneHotEncoder,Normalizer,Binarizer]

In [12]:
models = [
    ['KNeighborsRegressor',KNeighborsRegressor],
    ['LogisticRegression',LogisticRegression],
    ['LogisticRegressionCV',LogisticRegressionCV],
    ['DecisionTreeRegressor',DecisionTreeRegressor],
    ['GradientBoostingRegressor',GradientBoostingRegressor],
    ['AdaBoostRegressor',AdaBoostRegressor],
    ['RandomForestRegressor',RandomForestRegressor],
    ['BaggingRegressor',BaggingRegressor],
    ['GaussianNB',GaussianNB],
    ['ExtraTreesRegressor',ExtraTreesRegressor],
    ['CatBoost',CatBoost],
    ['CatBoostRegressor',CatBoostRegressor],
    ['XGBRegressor',XGBRegressor],
    ['XGBRFRegressor',XGBRFRegressor],
    ['ExtraTreesRegressor',ExtraTreesRegressor],
]

## Cleanning the data

In [13]:
X = data.drop('SalePrice',axis=1)
y = data['SalePrice']

In [14]:
str_cols = []
int_cols = []

In [15]:
for col_name,num_of_missing_rows,dtype in zip(list(X.columns),X.isna().sum(),X.dtypes):
    if dtype == object:
        str_cols.append(col_name)
    else:
        int_cols.append(col_name)

In [16]:
for str_col in str_cols:
    X,idx,labels_and_int_index,new_data = object_to_int(X,str_col)

In [17]:
X.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1,60,0,65.0,8450,0,0,0,0,0,...,0,0,0,0,0,0,2,2008,0,0
1,2,20,0,80.0,9600,0,0,0,0,0,...,0,0,0,0,0,0,5,2007,0,0
2,3,60,0,68.0,11250,0,0,1,0,0,...,0,0,0,0,0,0,9,2008,0,0
3,4,70,0,60.0,9550,0,0,1,0,0,...,0,0,0,0,0,0,2,2006,0,1
4,5,60,0,84.0,14260,0,0,1,0,0,...,0,0,0,0,0,0,12,2008,0,0


In [18]:
nan_cols = []
for col_name,num_of_missing_rows,dtype in zip(list(X.columns),X.isna().sum(),X.dtypes):
    if num_of_missing_rows > 0:
        nan_cols.append(col_name)

In [19]:
for nan_col in nan_cols:
    X[nan_col].fillna(X[nan_col].median(),inplace=True)

In [20]:
nan_cols = []
for col_name,num_of_missing_rows,dtype in zip(list(X.columns),X.isna().sum(),X.dtypes):
    if num_of_missing_rows > 0:
        nan_cols.append(col_name)

In [21]:
# train(GradientBoostingRegressor(),X,X,y,y,name='baseline-without-fe')

In [22]:
X_old = X.copy()

## FE

In [23]:
# for col_name in list(X.columns):
#     try:
#         X = X_old.copy()
#         X = fe(X,col_name)
#         train(GradientBoostingRegressor(),X,X,y,y,name=f'baseline-with-fe-{col_name}')
#     except:
#         print('*'*50)
#         print('*'*50)

In [24]:
# X = X_old.copy()

In [25]:
X_corr = X_old.corr()

In [26]:
keep_cols = []

## Data.corr()

In [27]:
# for key,val in zip(X_corr.to_dict().keys(),X_corr.to_dict().values()):
#     for val_key,val_vals in zip(val.keys(),val.values()):
#         if val_key == key:
#             pass
#         else:
#             if val_vals > 0.0:
#                 if val_key not in keep_cols:
#                     print(val_vals)
#                     keep_cols.append(val_key)

In [28]:
# fig,ax = plt.subplots(figsize=(25,12))
# ax = sns.heatmap(X_corr,annot=True,linewidths=0.5,fmt='.2f',cmap='YlGnBu')

In [29]:
# keep_cols

In [30]:
# len(keep_cols)

## Analytics

In [31]:
X.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1,60,0,65.0,8450,0,0,0,0,0,...,0,0,0,0,0,0,2,2008,0,0
1,2,20,0,80.0,9600,0,0,0,0,0,...,0,0,0,0,0,0,5,2007,0,0
2,3,60,0,68.0,11250,0,0,1,0,0,...,0,0,0,0,0,0,9,2008,0,0
3,4,70,0,60.0,9550,0,0,1,0,0,...,0,0,0,0,0,0,2,2006,0,1
4,5,60,0,84.0,14260,0,0,1,0,0,...,0,0,0,0,0,0,12,2008,0,0


## Preproccessing

In [32]:
X_old = X.copy()

In [33]:
# for preproccessing in preproccessings:
#     X = X_old.copy()
#     preproccessing = preproccessing()
#     X = preproccessing.fit_transform(X)
#     train(GradientBoostingRegressor(),X,X,y,y,name=f'{preproccessing}-preproccessing')

In [34]:
X = X_old.copy()

In [35]:
X = X_old.copy()

In [36]:
# X = feature_selection_prep_data(GradientBoostingRegressor(),X,y,select_from_model=False,variance_threshold=True,select_k_best=False,rfecv=False)
# train(GradientBoostingRegressor(),X,X,y,y,name=f'select_from_model=False-variance_threshold=True-select_k_best=False-rfecv=False-decomposition')

In [37]:
X = X_old.copy()

In [38]:
X = X_old.copy()

In [39]:
# X = feature_selection_prep_data(GradientBoostingRegressor(),X,y,select_from_model=False,variance_threshold=False,select_k_best=False,rfecv=True)
# train(GradientBoostingRegressor(),X,X,y,y,name=f'select_from_model=False-variance_threshold=False-select_k_best=False-rfecv=True-decomposition')

In [40]:
X = X_old.copy()

## Modelling

In [41]:
for model in models:
    train(model[1](),X,X,y,y,name=f'{model[0]}')

wandb: Currently logged in as: ranuga-d (use `wandb login --relogin` to force relogin)


mean_absolute_error,23969.50863
mean_squared_error,1422933124.74874
_runtime,6
_timestamp,1629882334
_step,1
val mean_absolute_error,23969.50863
val mean_squared_error,1422933124.74874


mean_absolute_error,▁
mean_squared_error,▁
_runtime,▁▁
_timestamp,▁▁
_step,▁█
val mean_absolute_error,▁
val mean_squared_error,▁


/home/indika/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


mean_absolute_error,37421.43288
mean_squared_error,3265112959.47808
_runtime,9
_timestamp,1629882365
_step,1
val mean_absolute_error,37421.43288
val mean_squared_error,3265112959.47808


mean_absolute_error,▁
mean_squared_error,▁
_runtime,▁▁
_timestamp,▁▁
_step,▁█
val mean_absolute_error,▁
val mean_squared_error,▁


/home/indika/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:668: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/home/indika/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/home/indika/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (m

ValueError: could not broadcast input array from shape (10,583,81) into shape (10)

## Random Search